In [1]:
#Import Packages
import ase
import os
from ase.io import read
import numpy as np
import csv
from ase.io.jsonio import read_json
import json
from scipy.stats import rankdata
from ase.visualize import view
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn.functional as F
from torch_geometric.data import DataLoader, Dataset, Data, InMemoryDataset
from torch_geometric.utils import dense_to_sparse, degree, add_self_loops
import torch_geometric.transforms as T
from torch_geometric.utils import degree

import glob, os

import networkx as nx

import trimesh
import pickle

In [2]:
def threshold_sort(matrix, threshold, neighbors, reverse=False, adj=False):
    mask = matrix > threshold
    distance_matrix_trimmed = np.ma.array(matrix, mask=mask)
    if reverse == False:
        distance_matrix_trimmed = rankdata(
            distance_matrix_trimmed, method="ordinal", axis=1
        )
    elif reverse == True:
        distance_matrix_trimmed = rankdata(
            distance_matrix_trimmed * -1, method="ordinal", axis=1
        )
    distance_matrix_trimmed = np.nan_to_num(
        np.where(mask, np.nan, distance_matrix_trimmed)
    )
    distance_matrix_trimmed[distance_matrix_trimmed > neighbors + 1] = 0

    if adj == False:
        distance_matrix_trimmed = np.where(
            distance_matrix_trimmed == 0, distance_matrix_trimmed, matrix
        )
        return distance_matrix_trimmed
    elif adj == True:
        adj_list = np.zeros((matrix.shape[0], neighbors + 1))
        adj_attr = np.zeros((matrix.shape[0], neighbors + 1))
        for i in range(0, matrix.shape[0]):
            temp = np.where(distance_matrix_trimmed[i] != 0)[0]
            adj_list[i, :] = np.pad(
                temp,
                pad_width=(0, neighbors + 1 - len(temp)),
                mode="constant",
                constant_values=0,
            )
            adj_attr[i, :] = matrix[i, adj_list[i, :].astype(int)]
        distance_matrix_trimmed = np.where(
            distance_matrix_trimmed == 0, distance_matrix_trimmed, matrix
        )
        return distance_matrix_trimmed, adj_list, adj_attr


In [7]:
# Loads MOFS Structure

max_nodes = 20

# os.chdir("./MOF_data")
mof_uc = []
i = 1
for file in glob.glob("C:\\Users\\GillA\\Desktop\\University\\PhD\\Projects\\Generation\\Zeolites\\qmof_database\\relaxed_structures\\*.cif"): # List of Cifs of files --> glob.glob looks for .json or *<>
    structure = ase.io.read(file) ### Relaxed Structure [No need for ]
    distance_matrix = structure.get_all_distances(mic=True) 
    num_of_nodes = distance_matrix.shape[0]
    if num_of_nodes <= max_nodes: # Only Files with less than 20 nodes
        mof_uc.append(structure)
    
    if i % 1000 == 0:
        print("Processed", i, "files")
        print("================================================")
    
    i = i + 1

Processed 1000 files
Processed 2000 files
Processed 3000 files
Processed 4000 files
Processed 5000 files
Processed 6000 files
Processed 7000 files
Processed 8000 files
Processed 9000 files
Processed 10000 files
Processed 11000 files
Processed 12000 files
Processed 13000 files
Processed 14000 files
Processed 15000 files
Processed 16000 files
Processed 17000 files
Processed 18000 files
Processed 19000 files
Processed 20000 files


In [9]:
unit_cell = []
mof_graph = []
for i in range(len(mof_uc)):
    print("Processing ",i,"-th graph", sep="")
    s1 = mof_uc[i]
    distance_matrix = s1.get_all_distances(mic=True)
    distance_matrix_trimmed = threshold_sort(distance_matrix,8,12,adj=False)
    distance_matrix_trimmed = torch.Tensor(distance_matrix_trimmed)
    distance_matrix_trimmed[distance_matrix_trimmed != 0] = 1
    # graph_tmp = nx.convert_matrix.from_numpy_matrix(distance_matrix_trimmed.numpy())
    mof_graph.extend([nx.convert_matrix.from_numpy_matrix(distance_matrix_trimmed.numpy())])
    unit_cell.extend([i])

Processing 0-th graph
Processing 1-th graph
Processing 2-th graph
Processing 3-th graph
Processing 4-th graph
Processing 5-th graph
Processing 6-th graph
Processing 7-th graph
Processing 8-th graph
Processing 9-th graph
Processing 10-th graph
Processing 11-th graph
Processing 12-th graph
Processing 13-th graph


In [10]:
nu = [nx.number_of_nodes(g) for g in mof_graph]

sum(nu)/len(nu)

18.928571428571427